In [1]:
from selenium import webdriver
from time import sleep

BASE_URL = "http://fitoleg.casafe.org/"
BUSQUEDAS = "busquedas/municipal"
USUARIO = "fitoleg"
CLAVE = "fitoleg"

In [2]:
driver = webdriver.Firefox()

In [3]:
driver.get(BASE_URL)

In [ ]:
def login():
    driver.get(f"{BASE_URL}/usuario/login")
    driver.find_element_by_id("usuario").send_keys(USUARIO)
    driver.find_element_by_id("password").send_keys(CLAVE + "\n")
    driver.find_element_by_id("btnLogin").click()    

In [ ]:
login()

In [ ]:
#
# Parsea Links
#

driver.get(f"{BASE_URL}/busquedas/municipal")

# Maximos items por pagina
driver.find_element_by_id("dropdownlistContentgridpagerlistjqxgrid").click()
sleep(1)
div = driver.find_element_by_id("listBoxContentinnerListBoxgridpagerlistjqxgrid")
div.find_elements_by_class_name("jqx-rc-all-classic")[-1].click()

vuelta_completa = False
links = []

while not vuelta_completa:
    # Iterar paginas
    
    while True:
        
        # Search links
        for link in driver.find_elements_by_tag_name("a"):
            href = link.get_attribute("href")
            
            if href[-4:] == ".pdf":
                if href not in links:
                    links.append(href)
    
        # Scroll down
        scroll = driver.find_element_by_id("jqxScrollAreaDownverticalScrollBarjqxgrid")
        if int(scroll.value_of_css_property("height")[:-2]) == 0:
            break
        else:
            driver.find_elements_by_class_name("jqx-icon-arrow-down")[-3].click()
            driver.find_elements_by_class_name("jqx-icon-arrow-down")[-3].click()
            #sleep(0.01)
    
    print(f"{len(links)} links")    
    
    pager = driver.find_element_by_id("pagerjqxgrid").text.split('\n')[0].replace("-", " ")
    pager = [x for x in pager.split() if x.isdigit()][1:]
    if pager[0] == pager[1]:
        break
        
    # 
    arrow = driver.find_elements_by_class_name("jqx-icon-arrow-right-classic")[-2]
    arrow.click()
    sleep(2)

In [6]:
#
# Parsea la tabla
#

driver.get(f"{BASE_URL}/busquedas/municipal")
driver.find_element_by_id("dropdownlistContentgridpagerlistjqxgrid").click()
sleep(1)
div = driver.find_element_by_id("listBoxContentinnerListBoxgridpagerlistjqxgrid")
div.find_elements_by_class_name("jqx-rc-all-classic")[-1].click()
recuadro = driver.find_element_by_id("contenttablejqxgrid")

filas = {}
pagina = 0
while True:
    
    pagina += 1
    
    while True:

        for div in recuadro.find_elements_by_tag_name("div"):
            if not div.is_displayed():
                continue
                
            div_id = div.get_attribute("id")
            if not div_id.startswith("row"):
                continue
            
            div_id = f"page{pagina}_{div_id}"
            if div_id in filas.keys():
                continue

            fila = []

            displayed = True
            for columna in div.find_elements_by_class_name("jqx-grid-cell")[1:]:                    
                fila.append(columna.text)

            filas[div_id] = fila

        # Scroll down
        scroll = driver.find_element_by_id("jqxScrollAreaDownverticalScrollBarjqxgrid")
        if int(scroll.value_of_css_property("height")[:-2]) == 0:
            break
        else:
            for c in range(10):
                driver.find_elements_by_class_name("jqx-icon-arrow-down")[-3].click()
                sleep(0.01)

    # check for last page
    pager = driver.find_element_by_id("pagerjqxgrid").text.split('\n')[0].replace("-", " ")
    pager = [x for x in pager.split() if x.isdigit()][1:]
    if pager[0] == pager[1]:
        break
        
    # next page
    arrow = driver.find_elements_by_class_name("jqx-icon-arrow-right-classic")[-2]
    arrow.click()
    sleep(0.5)

In [7]:
# should return same number of entries (340)
len(filas)

314

In [ ]:
# a monitor to check the list of lists
# bad news: notice that some fields are incomplete or blank
print(*filas[-1])

In [24]:
import csv
titles = ['---', 'Provincia', 'Municipio', 'Tipo', 'Descripción', 'Resumen']

with open('cuadro.csv', 'w') as f: 
# set unix dialect "field content", "field content" ... \n 
    write = csv.writer(f) 

    # you can customize a dialect:
    # write = csv.writer(f, quotechar='"', escapechar='\\', ... )
    write.writerow(titles)
    write.writerows([filas[x] for x in filas])

In [22]:
for x in filas:
    filas[x] = [x.replace("\n","") for x in filas[x]]

In [23]:
filas

{'page1_row0jqxgrid': ['Buenos Aires',
  'Ramallo',
  'Ordenanza',
  'Ordenanza 5133/15',
  'Distancia Aplicación:Aérea: Prohíbe Aplicación Aérea En Todo El Partido De RamalloTerrestre: Establece zona de exclusión 300 mts. y desde el limite de la zona de exclusion solo se podran aplicar productos banda verde o IV.Entra en vigencia 17 de Julio 2015'],
 'page1_row1jqxgrid': ['Buenos Aires',
  'Ramallo',
  'Ordenanza',
  'Ordenanza 4092/10',
  'Distancia Aplicación:Aérea: 1500 mts. de área urbanizada.Terrestre: 50 mts cortina forestal de 3 hileras de eucaliptus, alamos y casuarinas'],
 'page1_row2jqxgrid': ['Buenos Aires',
  'Exaltación de la Cruz',
  'Ordenanza',
  'Ordenanza 101/12',
  'Regula distancia 20 mts de cursos de agua. Regula envases vacíos.'],
 'page1_row3jqxgrid': ['Buenos Aires',
  'Rivadavia',
  'Ordenanza',
  'Ordenanza 3818/2016',
  'Prohibe todo el territorio del Partido de Rivadavia el uso y/o aplicación, transporte y comercialización del producto 2-4D, ya sea intensiv